In [1]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator
from sklearn.metrics import confusion_matrix
import seaborn as sns 
from sklearn.metrics import precision_score,recall_score,f1_score


# MNIST

there are 70000 images of size 28*28, each image has 784 features. 
each feature represents one pixel's intensity from 0 to 255 (should be white)

The MNIST dataset is already split to train set (60000) and test set (10000)

shuffle is important since some alg, are sensitive to the order of the train set and they perform poorly if they get many similar instances in a row 

In [ ]:
mnist_path = 'mnist-original.mat'

mnist = loadmat(mnist_path)
X = mnist['data'].T
y = mnist['label'][0]
display(X.shape , y.shape)

#peek a digit
some_index = 36000
some_digit = X[some_index]
display(some_digit.shape)
some_digit_image = some_digit.reshape(28,28)

plt.imshow(some_digit_image,cmap='binary')
plt.axis('off')

#show y value
display(y[36000])

In [ ]:
# split to train & test 
train_size = 60000
X_train,y_train = X[:train_size],y[:train_size]
X_test,y_test = X[train_size:],y[train_size:]


#shuffle
shuffle_index = np.random.permutation(train_size)
X_train,y_train = X[shuffle_index],y[shuffle_index]


# Train a Binary Classifier - By SGDClassifier
# Stochastic Gradient Descent
The class implements regularized linear classifier (SVM,logistic regression,etc) with SGD training
the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing learning rate. 

The model it fits can be controlled with loss parameter
by default, it fits a linear support vector machine (using the 'hinge' loss)
log loss gives logistic regression
perceptron is the linear loss used by the perceptron alg. 

The classifier has the advantage of being capable of handling very large dataset efficiently, and is also wekk suited for online learning

In [ ]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

display(y_train)
display(y_train_5, y_test_5)

In [ ]:
sgd_clf = SGDClassifier(random_state=0,max_iter=50)
sgd_clf.fit(X_train,y_train_5)

#display(some_digit)
sgd_clf.predict([some_digit])

# Performance Measure - cross_val_score()
evaluating a classifier is often significantly trickier than evaluating a regressor
a good start is to use cross-validation

the **cross_val_score()** return the ratio of correct predictions. 

~95% looks good but if we will write & check the Never5Classifier , we will get also 90% accuracy. 
This is because only about 10% of the images are 5s. so if you always guess than an image is not 5 , you will get right about 90% of the time

This demonstrates why accuracy is generally not the preferred performance measure for classifier, especially when you are dealing with skewed datasets (סט נתונים מוטה)

In [ ]:
cross_val_score(sgd_clf,X_train,y_train_5,cv=3,scoring='accuracy')



In [ ]:
class Never5Classifier(BaseEstimator):
    def fit(self,X,y=None):
        pass
    def predict(self,X):
        return np.zeros((len(X),1),dtype=bool)
        
never_5_clf = Never5Classifier()
cross_val_score(never_5_clf,X_train,y_train_5,cv=3,scoring='accuracy')

# Performance Measure - Confusion Matrix - cross_val_predict()

a much better way to evaluate the performance of a classifier is to look at the confusion matrix
The idea is to count the number of times instances of class A are classified as class B

for example, to know the number of times the classifier confused images of 5s with 3s, you would look in the 5th row and the 3rd column of the confusion matrix
To compute the confusion matrix, you first need to have a set of predictions

You could make predictions on the test set, but let's keep it untouched for now
you want to use yout test set only at the very end of your project, once you have a classifier you are ready to launch

Instead of, you can use the **cross_val_predict() **

Like the cross_val_score() the cross_val_predict() also performs K-cross validation, but instead of returning the evaluations scores, it returns for each element the prediction that was obtained for that element when it was in the test set
only cross validation strategies that assign all elements to a test set exactly once can be used (otherise  an exception is raised)


In [ ]:
y_train_pred = cross_val_predict(sgd_clf,X_train,y_train_5,cv=3)

conf_mat = confusion_matrix(y_train_5,y_train_pred)
conf_mat

In [ ]:
#visualize confusion matrix
sns.heatmap(conf_mat,square=True,annot=True,cmap='Blues',fmt='d',cbar=False)
plt.xlabel('Predicted Value',fontsize=12)
plt.ylabel('True Value',fontsize=12)

### accuracy / precision / recall (sensitivity or true positive rate TPR) / F1 score

In [ ]:
# TN FP
# FN TP

# accuracy = (TN + TP) / (TN + TP + FN + FP)
accuracy = (53177 + 4479) / (53177 + 1402 + 942 + 4479)
display('accuracy',accuracy)

# precision = (TP) / (TP + FP)
# accuracy of the positive prediction
precision = 4479 / (4479 + 1402)
display('precision',precision)

# recall = (TP) / (TP + FN)
# ratio of positive instances that are correctly detected 
recall = 4479 / (4479 + 942)
display('recall',recall)

In [ ]:
precision_score(y_train_5,y_train_pred) , recall_score(y_train_5,y_train_pred)
#0.76 - when it claims an image represent 5, it is correct only 76% (precision_score)
#0.82 - only detect 82% of the 5s (recall_score)

In [ ]:
# F1 = 2 / ( (1/precision) + (1/recall)) = 2 * ( (precision * recall) / (precision + recall) ) =  2TP / ( 2TP + FN + FP)
# מאחר שממוצע הרמוני של קבוצת מספרים נוטה לעבר המספר הקטן ביותר שבה, הוא נוטה למזער את ההשפעה של מספרים גדולים ולהגדיל את ההשפעה של מספרים קטנים.

# The harmonic mean give much more weight to low values
# Thus, a classifier will only get a high F1 score if both recall and precision are high

f1_score(y_train_5,y_train_pred)


In [2]:
#Ex F1
d = {'expected':[1,1,0,1,0,0,1,0,0,0],
    'predicted':[1,0,0,1,0,0,1,1,1,0]}

df = pd.DataFrame(d)

In [15]:
#confusion matrix by hand
# TN - 4 ex 0 pre 0 (4) FP ex 0 pre 1 (2)  
# FN - 2 ex 1 pre 0 (1) TP ex 1 pre 1 (3) 
y_expected =  df['expected']
y_predicted = df['predicted']

conf_mat_ex = confusion_matrix(y_expected,y_predicted)
display(conf_mat_ex)

display(precision_score(y_expected,y_predicted) , recall_score(y_expected,y_predicted))
f1_score(y_expected,y_predicted)


array([[4, 2],
       [1, 3]], dtype=int64)

0.6

0.75

0.6666666666666665